# Diffusion Model met Fashion MNIST in PyTorch

Het doel van dit notebook is om een diffusion model te implementeren dat leert om ruis van afbeeldingen te verwijderen en nieuwe afbeeldingen van mode-items te genereren die vergelijkbaar zijn met de Fashion MNIST dataset.

## Importeren van packages en dataset

In onderstaande code worden de volgende stappen uitgevoerd:
* Laad de Fashion MNIST dataset en normaliseer de afbeeldingen naar het bereik [-1, 1]
* Maak een DataLoader om batches van trainingsgegevens te genereren
* Voeg functies toe om eenvoudiger te kunnen werken met ruis

In [ ]:
# Importeren van benodigde bibliotheken
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import math
import os

# Controleer of er een GPU beschikbaar is, zo niet gebruik de CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data-transformatie: normaliseer de afbeeldingen zodat de pixelwaarden tussen -1 en 1 liggen
transform = transforms.Compose([
    transforms.ToTensor(),  # Converteert beeld naar tensor
])

# FashionMNIST dataset downloaden en laden
train_dataset = datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)

# DataLoader voor batches van de trainingsdata
train_loader = DataLoader(dataset=train_dataset, batch_size=8, shuffle=True)

print("Data geladen en DataLoader klaar.")

x, y = next(iter(train_loader))
print('Input shape:', x.shape)
print('Labels:', y)
plt.imshow(torchvision.utils.make_grid(x)[0], cmap='Greys')

## Bouwen van een model

In deze notebook gaan we werken met een eenvoudig U-Net model dat de taak van ruisvoorspelling uitvoert. Het model bestaat uit encoder-, midden- en decoderlagen. Het model voorspelt de ruis die is toegevoegd aan een afbeelding en probeert deze te verwijderen.
Ook andere modellen kunnen hiervoor gebruikt worden.

## Trainen van het model

Om het model te trainen wordt de verliesfunctie MSE-loss en de Adam-optimizer gebruikt. 
Daarnaast berekenen we de betas voor het diffusieproces met behulp van een lineaire verdeling over 1000 timesteps.

Ten slotte wordt het diffusiemodel getraind. Voor elke batch van trainingsafbeeldingen wordt een willekeurige tijdstap gekozen om ruis toe te voegen aan de afbeelding. Het model leert vervolgens de toegevoegde ruis te voorspellen en te verwijderen. Voor elke epoch wordt het verlies uitgeprint.

## Visualisatie

### Opmerkingen:

De hierboven gevolgde aanpak is een vereenvoudigde voorstelling van de state-of-the-art diffusion model die wel goed de te volgen concepten uitlegt.
Een meer gedetailleerde uitleg van diffusion models vind je [DDPM paper](https://arxiv.org/abs/2006.11239)
In deze paper breiden ze het voorbeeld in deze notebook uit door:
* Een geavanceerder UNet-model te gebruiken
* Een complexere manier om ruis toe te voegen
* Niet rechtstreeks het begin image te voorspellen maar de ruis in een image te voorspellen
* Gebruik een timestep in plaats van een amount-waarde